In [1]:
import pandas as pd
r1 = pd.read_csv('pse_isr_reddit_comment.csv')
r2 = pd.read_csv('reddit_opinion_PSE_ISR.csv')

reddit = pd.concat((r2.iloc[:,0:5], r1), axis=0)

In [6]:
r1.created_time= pd.to_datetime(r1.created_time)

In [7]:
r2.created_time= pd.to_datetime(r2.created_time)

In [15]:
r2[r2.created_time < '2023-10-01'].shape

(1992, 24)

In [22]:
reddit.head()

,comment_id,score,self_text,subreddit,created_time
0,l0me1zt,1,"Arabs can buy land in Israel today, under the ...",IsraelPalestine,2024-04-21 17:23:31
1,l0me0k3,1,"&gt; My bad then, but a major reason as to why...",IsraelPalestine,2024-04-21 17:23:17
2,l0mdz6a,1,Oooo nice! I mean someone should make use of t...,IsrealPalestineWar_23,2024-04-21 17:23:03
3,l0mdya0,1,They can try to get a piece from Turkey.,AskMiddleEast,2024-04-21 17:22:54
4,l0mdxs5,1,You made the right decision.,ActualPublicFreakouts,2024-04-21 17:22:50


In [23]:
reddit_dataset_sample = reddit.sample(frac = 1, random_state = 42)

In [24]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

def preprocessing(corpus):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    corpus = re.sub(r'[^\w\s]', '', corpus)
    tokens = word_tokenize(corpus)
    tokens = [word.lower() for word in tokens]
    tokens = [word.replace(',' , '') for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

In [28]:
corpus_processed = [preprocessing(corpus) for corpus in reddit_dataset_sample.self_text.astype(str).values]

In [29]:
reddit_dataset_sample['corpus_processed'] = corpus_processed

In [30]:
from gensim.models import Word2Vec
from gensim.models.word2vec import FAST_VERSION
from multiprocessing import cpu_count

# initialize model

w2v_model = Word2Vec(vector_size=100, # vector size
                     window=10, # window for sampling
                     sample=0.01, # subsampling rate
                     epochs=50, # iterations
                     negative=10, # negative samples
                     min_count=10, # minimum threshold
                     workers=cpu_count(), # parallelize to all cores
                     hs=0, # no hierarchical softmax when negative sampling is active
)

# build the vocabulary
w2v_model.build_vocab(corpus_processed)

# train the model
w2v_model.train(corpus_processed,
                total_examples=w2v_model.corpus_count,
                epochs=w2v_model.epochs)


(2094237150, 2130400500)

In [43]:
model_total.save('model_total')

### Overtime Analysis

In [37]:
reddit_dataset_sample['created_time'] = pd.to_datetime(reddit_dataset_sample['created_time'], utc=True)

reddit_dataset_sample['month'] = reddit_dataset_sample['created_time'].dt.strftime('%Y-%B')

# Dictionary to hold the dataframes by month
df_dict = {}
for month in reddit_dataset_sample['month'].unique():
    df_dict[month] = reddit_dataset_sample[reddit_dataset_sample['month'] == month]

In [38]:
reddit_dataset_sample.to_csv('reddit_dataset_final_processed.csv')

In [39]:
from gensim.models import Word2Vec
from gensim.models.word2vec import FAST_VERSION
from multiprocessing import cpu_count


# initialize model

modellist = []


for month, corpus in df_dict.items():
    print(f'doing {month}')
    corpus_processed = corpus['corpus_processed']

    w2v_model = Word2Vec(vector_size=100,
                        window=10, 
                        sample=0.01, 
                        epochs=50, 
                        negative=10, 
                        min_count=10, 
                        workers=cpu_count(), 
                        hs=0, 
    )

    # build the vocabulary
    w2v_model.build_vocab(corpus_processed)

    # train the model
    w2v_model.train(corpus_processed,
                    total_examples=w2v_model.corpus_count,
                    epochs=w2v_model.epochs)
    modellist.append(w2v_model)


doing 2023-November
doing 2024-January
doing 2023-October
doing 2024-March
doing 2023-December
doing 2024-April
doing 2024-February
doing 2023-September


In [ ]:
import pickle   
with open('model_list', 'wb') as f:
    pickle.dump(modellist, f) 

